# Synthesize a research paper
## Workflow
+ Define domain<br>
+ Build corpus<br>
+ Create embeddings model<br>
+ Perform WhiteSpace Analysis<br>
+ Identify best area for analysis<br>
+ Suggest topics for research<br>
+ Synthesize research paper content on selected topic

## Import Libraries

In [1]:
import os
import time

from arxiv import SortCriterion, SortOrder, Client, Search
import csv

import openai

import pdfplumber
import pytesseract

# Inputs

In [2]:
# Define domain
domain = "petroleum engineering"

In [3]:
# Model definitions
# openAI key
openai.api_key = "fb2a1f252f6547fcad7432ecd52cb56d"
# for azure openAI
# your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "https://ics-rd-openai.openai.azure.com/" 
openai.api_version = '2022-12-01' # this may change in the future
openai.api_type = 'azure'
#This will correspond to the custom name you chose for your deployment when you deployed a model. 
deployment_name='CVX_Davinci_003' 

# misc definitions
corpus_directory = "./corpus"
max_papers = 100
train_filename = "train.txt"

## Helper functions

In [4]:
# create a function to generate key words for research in the defined domain
def generate_research_keywords_for_domain(domain:str)->str:
    prompt = "Create comma seperated key words for research queries in the "+domain+" domain:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.20,
            max_tokens=500
        )
    result = response.choices[0].text.strip()
    return result

def string_to_list(text:str)->str:
    """
    Converts a string of comma-separated values into a list of individual elements.
    """
    # Split the string into individual values using the comma separator
    values = text.split(',')
    
    # Remove any whitespace from the values
    values = [value.strip().replace(".", "") for value in values]
    
    # Return the list of values
    return values

def create_directory_if_not_exists(path:str):
    """
    Creates a directory at the specified path if it does not already exist.
    """
    if not os.path.exists(path):
        os.makedirs(path)
    return

# create a function to save papers by removing special characters from the file name
def clean_title(title:str):
    title = title.replace(" ", "_")
    title = title.replace("\\", "")
    title = title.replace("/", "")
    return title

def file_exists(path:str):
    """
    Checks if a file exists at the specified path.
    Returns True if the file exists, False otherwise.
    """
    return os.path.exists(path) and os.path.isfile(path)

def get_typed_files_in_directory(directory:str, file_type:str) -> list:
    """
    Returns a list of all files in the specified directory.
    """
    # Initialize an empty list to hold the PDF files
    typed_files = []

    # Loop over all files in the directory
    for filename in os.listdir(directory):
        # Check if the file has a .pdf extension
        if filename.lower().endswith(f".{file_type.lower()}"):
            # If it's a PDF file, add the full path to the list
            pdf_files.append(os.path.join(directory, filename))

    # Return the list of PDF files
    return pdf_files

def write_or_append_file(filepath, text):
    """
    Writes the specified text to a file, creating the file if it does not exist,
    or appends the text to the end of the file if it already exists.
    """
    mode = "a" if os.path.exists(filepath) else "w"
    with open(filepath, mode) as f:
        f.write(text)
        
def extract_text_from_pdf(filepath:str)->str:
    """
    Extracts text from a PDF file, using OCR if the PDF contains images.
    """
    # Open the PDF file using pdfplumber
    with pdfplumber.open(filepath) as pdf:
        # Initialize an empty string to hold the extracted text
        text = ""

        # Loop over all pages in the PDF
        for page in pdf.pages:
            # Extract the page text using pdfplumber
            page_text = page.extract_text()

            # If the page text is None, the page may contain images, so use OCR to extract the text
            if page_text is None:
                # Convert the page image to grayscale and apply thresholding to remove noise
                page_image = page.to_image(resolution=150)
                page_image = page_image.convert('L')
                page_image = page_image.point(lambda x: 0 if x < 180 else 255, '1')

                # Use pytesseract to extract the text from the image
                page_text = pytesseract.image_to_string(page_image)

            # Add the page text to the overall text
            text += page_text

    # Return the extracted text
    return text

## Build corpus
Here we will build a corpus of documents from ARXIV papers<br>
To search papers, generate a list of keywords of research areas in the defined domain

In [5]:
keywords = generate_research_keywords_for_domain(domain)
keywords = string_to_list(keywords)

In [6]:
keywords

['petroleum engineering',
 'oil exploration',
 'drilling',
 'reservoir engineering',
 'production engineering',
 'well completion',
 'well logging',
 'geology',
 'seismic surveying',
 'petroleum economics',
 'petroleum refining',
 'petroleum geology',
 'petroleum technology',
 'petroleum safety',
 'petroleum law',
 'environmental impact',
 'offshore drilling',
 'hydraulic fracturing']

In [ ]:
# download papers in each research area
create_directory_if_not_exists(corpus_directory)
for searchterm in keywords:
    # create arxiv client
    big_slow_client = Client(
      page_size = 1000,
      delay_seconds = 10,
      num_retries = 5
    )
    for result in big_slow_client.results(Search(query=searchterm, max_results = max_papers, sort_by = SortCriterion.Relevance, sort_order = SortOrder.Ascending)):
        filename = clean_title(result.title)+".pdf"
        filepath = os.path.join(corpus_directory, filename)
        try:
            if file_exists(filepath):
                print("downloaded paper: "+filepath)
            else:
                print("downloading paper: "+filepath)
                result.download_pdf(dirpath=corpus_directory, filename=filename)
                time.sleep(2)
        except:
            print("Error downloading paper: "+filename)
            time.sleep(10)
            continue

downloaded paper: ./corpus/Optimal_Economic_Operation_of_Liquid_Petroleum_Products_Pipeline_Systems.pdf
downloaded paper: ./corpus/Phase_behavior_in_petroleum_fluids_(A_Detailed_Descriptive_and_Illustrative_Account).pdf
downloaded paper: ./corpus/On_the_Determination_of_the_Solar_Rotation_Elements_i,_Ω_and_Period_using_Sunspot_Observations_by_Ruđer_Bošković_in_1777.pdf
downloaded paper: ./corpus/On_gray-box_modeling_for_virtual_flow_metering.pdf
downloaded paper: ./corpus/Reaction_Mechanisms_in_Petroleum:_From_Experimentation_to_Upgrading_and_Geological_Conditions.pdf
downloaded paper: ./corpus/Simulation_of_incompressible_two-phase_flow_in_porous_media_with_large_timesteps.pdf
downloaded paper: ./corpus/Multi-task_learning_for_virtual_flow_metering.pdf
downloaded paper: ./corpus/Forecasting_the_production_of_Distillate_Fuel_Oil_Refinery_and_Propane_Blender_net_production_by_using_Time_Series_Algorithms.pdf
downloaded paper: ./corpus/A_Decision_Support_System_for_Multi-target_Geosteeri

downloaded paper: ./corpus/Application_of_Probabilistic_Graphical_Models_in_Forecasting_Crude_Oil_Price.pdf
downloaded paper: ./corpus/Experimental_evaluation_of_a_silicone_oil_as_an_oxidation_inhibitor_for_magnesium_alloy_under_contact_sliding_at_elevated_temperatures.pdf
downloaded paper: ./corpus/Exploring_the_use_of_Transition_Path_Theory_in_building_an_oil_spill_prediction_scheme.pdf
downloaded paper: ./corpus/Exploration_of_Spanish_Olive_Oil_Quality_with_a_Miniaturized_Low-Cost_Fluorescence_Sensor_and_Machine_Learning_Techniques.pdf
downloaded paper: ./corpus/Olive_Oil_is_Made_of_Olives,_Baby_Oil_is_Made_for_Babies:_Interpreting_Noun_Compounds_using_Paraphrases_in_a_Neural_Model.pdf
downloaded paper: ./corpus/Stability_of_additive-free_water-in-oil_emulsions.pdf
downloaded paper: ./corpus/Pickering_emulsions_with_alpha-cyclodextrin_inclusions:_Structure_and_thermal_stability.pdf
downloaded paper: ./corpus/Physico-chemical_properties_extraction_from_the_fluorescence_spectrum_with_

downloading paper: ./corpus/Mean_Field_Game_Approach_to_Production_and_Exploration_of_Exhaustible_Commodities.pdf
downloaded paper: ./corpus/On_the_short-term_influence_of_oil_price_changes_on_stock_markets_in_GCC_countries:_linear_and_nonlinear_analyses.pdf
downloaded paper: ./corpus/Temperature_Effect_on_Interactions_of_Oil_Droplet_with_Water-wetted_Shale_Kerogen_at_Reservoir_Temperatures:_Linear_Relationships_between_Temperature,_Free_Energy,_and_Contact_Angle.pdf
downloaded paper: ./corpus/Effects_of_crossflow_velocity_and_transmembrane_pressure_on_microfiltration_of_oil-in-water_emulsions.pdf
downloaded paper: ./corpus/Qualitative_detection_of_oil_adulteration_with_machine_learning_approaches.pdf
downloaded paper: ./corpus/Experimental_Investigation_of_the_Effects_of_Gas_Oil_and_Benzene_on_the_Geotechnical_Properties_of_Sandy_Soils.pdf
downloaded paper: ./corpus/Quantitative_Assessment_of_Adulteration_and_Reuse_of_Coconut_Oil_Using_Transmittance_Multispectral_Imaging.pdf
downloade

downloaded paper: ./corpus/Maximal_Steered_Coherence_Protection_by_Quantum_Reservoir_Engineering.pdf
downloaded paper: ./corpus/Extracting_work_from_random_collisions:_A_model_of_a_quantum_heat_engine.pdf
downloaded paper: ./corpus/Designing_reservoirs_for_1t_decoherence_of_a_qubit.pdf
downloaded paper: ./corpus/Carnot's_theorem_for_nonthermal_stationary_reservoirs.pdf
downloaded paper: ./corpus/A_Micrometer-sized_Heat_Engine_Operating_Between_Bacterial_Reservoirs.pdf
downloaded paper: ./corpus/Efficiency_of_heat_engines_coupled_to_nonequilibrium_reservoirs.pdf
downloaded paper: ./corpus/Optimization_performance_of_quantum_Otto_heat_engines_and_refrigerators_with_squeezed_thermal_reservoirs.pdf
downloaded paper: ./corpus/Quantitative_supply_security_related_significance_measures_for_gas_reservoires.pdf
downloaded paper: ./corpus/Electromagnetically_Induced_Transparency_and_Quantum_Heat_Engines.pdf
downloaded paper: ./corpus/The_thermodynamics_governing_'endoreversible'_engines.pdf
down

downloaded paper: ./corpus/A_Business_Maturity_Model_of_Software_Product_Line_Engineering.pdf
downloaded paper: ./corpus/An_Architecture_Process_Maturity_Model_of_Software_Product_Line_Engineering.pdf
downloaded paper: ./corpus/Variability_and_Evolution_in_Systems_of_Systems.pdf
downloaded paper: ./corpus/Towards_a_Systems_Engineering_based_Automotive_Product_Engineering_Process.pdf
downloaded paper: ./corpus/Towards_a_Systems_Engineering_Essence.pdf
downloaded paper: ./corpus/Mind_the_Gap:_On_the_Relationship_Between_Automatically_Measured_and_Self-Reported_Productivity.pdf
downloaded paper: ./corpus/Resolving_code_smells_in_software_product_line_using_refactoring_and_reverse_engineering.pdf
downloaded paper: ./corpus/Fault-Tolerant_Dot-Product_Engines.pdf
downloaded paper: ./corpus/Data_Engineering_for_the_Analysis_of_Semiconductor_Manufacturing_Data.pdf
downloaded paper: ./corpus/Optimization_analysis_of_an_endoreversible_quantum_heat_engine_with_efficient_power_function.pdf
downloa

downloaded paper: ./corpus/Completions_of_Countable_Excellent_Domains_and_Countable_Noncatenary_Domains.pdf
downloaded paper: ./corpus/Generalized_existential_completions_and_their_regular_and_exact_completions.pdf
downloaded paper: ./corpus/Uniqueness_of_Instantaneously_Complete_Ricci_flows.pdf
downloaded paper: ./corpus/A_note_on_the_statistical_view_of_matrix_completion.pdf
downloaded paper: ./corpus/A_completeness_result_for_implicit_justification_stit_logic.pdf
downloaded paper: ./corpus/On_well-dominated_direct,_Cartesian_and_strong_product_graphs.pdf
downloaded paper: ./corpus/Dedekind_complete_and_order_continuous_Banach_$C(K)$-modules.pdf
downloaded paper: ./corpus/Smooth_prime_Fano_complete_intersections_in_toric_varieties.pdf
downloaded paper: ./corpus/Profinite_completions_and_MacNeille_completions_of_MV-algebras.pdf
downloaded paper: ./corpus/On_Stoltenberg's_quasi-uniform_completion.pdf
downloaded paper: ./corpus/Action_Completion:_A_Temporal_Model_for_Moment_Detection.pd

downloading paper: ./corpus/Dynamic_and_Multi-functional_Labeling_Schemes.pdf
downloading paper: ./corpus/Disciplined_Geometric_Programming.pdf
downloading paper: ./corpus/Random_input_helps_searching_predecessors.pdf
downloading paper: ./corpus/Large_gaps_between_primes.pdf
downloading paper: ./corpus/Hyperbolicity_for_log_smooth_families_with_maximal_variation.pdf
downloading paper: ./corpus/The_Log_Product_Formula.pdf
downloading paper: ./corpus/Higher_order_corrections_for_anisotropic_bootstrap_percolation.pdf
Error downloading paper: Higher_order_corrections_for_anisotropic_bootstrap_percolation.pdf
downloading paper: ./corpus/Near-Optimal_(Euclidean)_Metric_Compression.pdf
Error downloading paper: Near-Optimal_(Euclidean)_Metric_Compression.pdf
downloading paper: ./corpus/Improved_Parallel_Construction_of_Wavelet_Trees_and_RankSelect_Structures.pdf
downloaded paper: ./corpus/On_the_Profile_of_Multiplicities_of_Complete_Subgraphs.pdf
downloading paper: ./corpus/HyperLogLogLog:_Car

In [ ]:
# get a list of files downloaded to the directory to process
pdf_file_list = get_typed_files_in_directory(corpus_directory, "pdf")
# extract the text from the pdf files
for file in pdf_file_list:
    print(f"processing file {file}")
    text = extract_text_from_pdf(file)
    if text is not None and len(text) > 10:
        write_or_append_file(train_filename, text)
    else:
        print(f"non text extract from file {file}")